In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from scipy.stats import chisquare
from scipy.stats import chi2_contingency

pd.set_option('display.max_columns', 100)

# Part 1
## *Data exploration and manipulation*
### Exploratory data analysis

In [3]:
train = pd.read_csv('C:/Users/Owner/Documents/Personal_Things/Springboard/Capstone1/train.csv')
test = pd.read_csv('C:/Users/Owner/Documents/Personal_Things/Springboard/Capstone1/test.csv')
sample_sub = pd.read_csv('C:/Users/Owner/Documents/Personal_Things/Springboard/Capstone1/sample_submission.csv')

# Exploratory Data Analysis (EDA)

In [3]:
train.head(10)

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0
5,19,0,5,1,4,0,0,0,0,0,1,0,0,0,0,0,6,1,0,0,0.9,1.8,2.332649,10,0,-1,0,0,14,1,1,0,1,104,2,0.445982,0.879049,0.406202,3.000000,0.7,0.8,0.4,3,1,8,2,11,3,8,4,2,0,9,0,1,0,1,1,1
6,20,0,2,1,3,1,0,0,1,0,0,0,0,0,0,0,8,1,0,0,0.6,0.1,0.617454,6,1,-1,0,1,11,1,1,0,1,99,2,0.316228,0.639683,0.368782,3.162278,0.2,0.6,0.5,2,2,8,1,8,3,10,3,0,0,10,0,1,0,0,1,0
7,22,0,5,1,4,0,0,1,0,0,0,0,0,0,0,0,13,1,0,0,0.7,0.4,0.607248,11,1,0,0,0,11,1,1,2,1,30,3,0.446990,0.900574,0.374833,3.316625,0.1,0.5,0.1,1,2,7,1,6,1,13,7,1,3,6,1,0,1,0,1,0
8,26,0,5,1,3,1,0,0,0,1,0,0,0,0,0,0,6,1,0,0,0.9,0.7,0.901388,10,1,-1,0,1,14,1,1,0,1,68,3,0.400000,0.780641,0.407431,2.828427,0.9,0.8,0.6,3,1,7,3,9,4,11,4,2,1,5,0,1,0,0,0,1
9,28,1,1,1,2,0,0,0,1,0,0,0,0,0,0,0,4,0,0,1,0.9,1.4,2.316652,11,0,0,1,0,14,1,1,2,1,104,2,0.447214,1.458184,0.390256,3.605551,0.7,0.8,0.8,2,2,8,2,9,1,11,3,5,0,6,0,1,0,0,1,0


We can see the target column is a 0 for if the customer did not file an insurance claim, or 1 for if the customer did file an insurance claim. 

Then let us count the feature columns by their categories and also tag their types:

ps_ind = 18 columns (ALL int)
   - ps_ind_cat = 02, 04, 05 (3 total)
   - ps_ind_bin = 06, 07, 08, 09, 10, 11, 12, 13, 16, 17, 18 (11 total)
   - ps_ind = 01, 03, 14, 15 (4 total)
    
ps_reg = 3 columns (all float)

ps_car = 15 columns (mixed types)
   - ps_car_cat = 01 -> 11 (11 total, all int)
   - ps_car = 11, 12, 13, 14, 15 (5 total, all float except 11 is int)
   - this is strange, ps_car_11 appears twice: once as categorical and once as int
    
ps_calc = 20 columns (mixed types)
   - ps_calc = 14 columns (01, 02, 03 are float while 04 -> 14 are int)
   - ps_calc_bin = 15 -> 20 (6 total, all int)
    
ind = individual      
reg = regulatory      
car = automobile score     
calc = calculation

In [4]:
train.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.012451,7.299922,0.660823,0.121081,0.153446,0.610991,0.439184,0.551102,8.295933,0.829931,-0.504899,0.725192,-0.157732,6.555340,0.910027,0.832080,1.328890,0.992136,62.215674,2.346072,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.127545,3.546042,0.473430,0.326222,0.360417,0.287643,0.404264,0.793506,2.508270,0.375716,0.788654,2.153463,0.844417,5.501445,0.347106,0.373796,0.978747,0.091619,33.012455,0.832548,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.525000,7.000000,1.000000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.720677,7.000000,1.000000,-1.000000,0.000000,0.000000,7.0000

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
id                595212 non-null int64
target            595212 non-null int64
ps_ind_01         595212 non-null int64
ps_ind_02_cat     595212 non-null int64
ps_ind_03         595212 non-null int64
ps_ind_04_cat     595212 non-null int64
ps_ind_05_cat     595212 non-null int64
ps_ind_06_bin     595212 non-null int64
ps_ind_07_bin     595212 non-null int64
ps_ind_08_bin     595212 non-null int64
ps_ind_09_bin     595212 non-null int64
ps_ind_10_bin     595212 non-null int64
ps_ind_11_bin     595212 non-null int64
ps_ind_12_bin     595212 non-null int64
ps_ind_13_bin     595212 non-null int64
ps_ind_14         595212 non-null int64
ps_ind_15         595212 non-null int64
ps_ind_16_bin     595212 non-null int64
ps_ind_17_bin     595212 non-null int64
ps_ind_18_bin     595212 non-null int64
ps_reg_01         595212 non-null float64
ps_reg_02         595212 non-null float64
ps_re

In [6]:
train.shape

(595212, 59)

In [7]:
train.drop_duplicates()
train.shape

(595212, 59)

There are no duplicate entries.

In [8]:
#Counting how many people filed an insurance claim by percentage out of 595,212 people, target = 1
print('% of people who filed insurance claim: ' + str(np.mean(train['target'])*100))

% of people who filed insurance claim: 3.6447517859182947


In [9]:
#Looking at test data now
print(test.shape)
test.drop_duplicates()
print(test.shape)

(892816, 58)
(892816, 58)


In [10]:
test.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610328,7,1,-1,0,-1,1,1,1,2,1,65,1,0.316228,0.669556,0.352136,3.464102,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362,4,1,-1,0,0,11,1,1,0,1,103,1,0.316228,0.606320,0.358329,2.828427,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0.4,0.0,0.916174,11,1,-1,0,-1,14,1,1,2,1,29,3,0.400000,0.896239,0.398497,3.316625,0.6,0.6,0.6,2,3,7,4,6,3,12,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,0,0,0,0,0,4,1,0,0,0.1,0.2,-1.000000,7,1,-1,0,-1,1,1,1,2,1,40,2,0.374166,0.652110,0.381445,2.449490,0.1,0.5,0.5,2,1,7,3,12,1,13,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,0,0,0,0,0,4,1,0,0,0.9,0.4,0.817771,11,1,-1,0,-1,11,1,1,2,1,101,3,0.374166,0.812914,0.385097,3.316625,0.9,0.6,0.8,3,4,7,1,10,4,12,4,0,0,4,0,1,1,0,0,1


There is no target column in the test data, this makes sense because we will use this data to predict the target, however, we won't be able to test if our predictions are accurate

In [11]:
test.describe()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,8.928160e+05,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.00000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000,892816.000000
mean,7.441535e+05,1.902371,1.358613,4.413734,0.417361,0.408132,0.393246,0.257191,0.163659,0.185905,0.000373,0.001595,0.009376,0.001039,0.012383,7.296862,0.660591,0.120399,0.154973,0.611057,0.439868,0.551740,8.292038,0.82994,-0.504895,0.725798,-0.158449,6.564297,0.910144,0.832311,1.327911,0.992134,62.284293,2.346606,0.379957,0.813637,0.276431,3.068325,0.449632,0.450453,0.450054,2.371390,1.885328,7.687805,3.009537,9.225854,2.338538,8.443167,5.438478,1.440265,2.875013,7.540367,0.123720,0.627756,0.554660,0.287796,0.349344,0.152428
std,4.296830e+05,1.986503,0.663002,2.700149,0.493453,1.355068,0.488471,0.437086,0.369966,0.389030,0.019309,0.039905,0.096375,0.032223,0.127251,3.541948,0.473509,0.325428,0.361879,0.287699,0.404678,0.794023,2.508331,0.37570,0.788753,2.154886,0.844566,5.500485,0.347283,0.373590,0.978741,0.091530,33.003714,0.831666,0.058385,0.224779,0.356835,0.729005,0.287213,0.287373,0.287254,1.116953,1.136763,1.333521,1.415155,1.460560,1.248601,2.906361,2.330081,1.200620,1.694072,2.745882,0.329262,0.483403,0.497004,0.452736,0.476763,0.359435
min,0.000000e+00,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.00000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,1.000000,-1.000000,0.141421,0.275778,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.720218e+05,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.400000,0.200000,0.524404,7.000000,1.00000,-1.000000,0.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,32.000000,2.000000,0.316228,0.671234,0.333916,2.828427,0.200000,0.200000,0.200000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.443070e+05,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,0.700000,0.300000,0.721110,7.000000,1.00000,-1.000000,0.000000,0.000000,7.000000,1.000000,1.000000,2.000000,1.000000,65.000000,3.000000,0.374166,0.7660

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892816 entries, 0 to 892815
Data columns (total 58 columns):
id                892816 non-null int64
ps_ind_01         892816 non-null int64
ps_ind_02_cat     892816 non-null int64
ps_ind_03         892816 non-null int64
ps_ind_04_cat     892816 non-null int64
ps_ind_05_cat     892816 non-null int64
ps_ind_06_bin     892816 non-null int64
ps_ind_07_bin     892816 non-null int64
ps_ind_08_bin     892816 non-null int64
ps_ind_09_bin     892816 non-null int64
ps_ind_10_bin     892816 non-null int64
ps_ind_11_bin     892816 non-null int64
ps_ind_12_bin     892816 non-null int64
ps_ind_13_bin     892816 non-null int64
ps_ind_14         892816 non-null int64
ps_ind_15         892816 non-null int64
ps_ind_16_bin     892816 non-null int64
ps_ind_17_bin     892816 non-null int64
ps_ind_18_bin     892816 non-null int64
ps_reg_01         892816 non-null float64
ps_reg_02         892816 non-null float64
ps_reg_03         892816 non-null float64
ps_

We can see that the number of columns in the test file is identical to the train file, minus the target column as was mentioned to be missing before. 

We will need to deal with missing values, which we know to be marked with "-1" before we do anything else with the train and test data

### Finding and replacing missing values

In [13]:
for col in train.columns:
    col_count = train[train[col] == -1][col].count()
    ratio = col_count/train.shape[0]
    if ratio > .35:
        print('\033[1;31;47m {} contains {} missing values which is {:.2%} of the total train records'.format(col, col_count,ratio))
    elif col_count > 0:
        print('\033[0;37;40m {} contains {} missing values which is {:.2%} of the total train records'.format(col, col_count,ratio))
    


 ps_ind_02_cat contains 216 missing values which is 0.04% of the total train records
 ps_ind_04_cat contains 83 missing values which is 0.01% of the total train records
 ps_ind_05_cat contains 5809 missing values which is 0.98% of the total train records
 ps_reg_03 contains 107772 missing values which is 18.11% of the total train records
 ps_car_01_cat contains 107 missing values which is 0.02% of the total train records
 ps_car_02_cat contains 5 missing values which is 0.00% of the total train records
 ps_car_03_cat contains 411231 missing values which is 69.09% of the total train records
 ps_car_05_cat contains 266551 missing values which is 44.78% of the total train records
 ps_car_07_cat contains 11489 missing values which is 1.93% of the total train records
 ps_car_09_cat contains 569 missing values which is 0.10% of the total train records
 ps_car_11 contains 5 missing values which is 0.00% of the total train records
 ps_car_12 contains 1 missing values which is 0.00% of the tota

ps_car_03_cat and ps_car_05_cat contain 69.09% and 44.78% missing values, so we should remove them entirely from consideration.  

When imputing, I believe we should fill in missing data first by its data type, then by its percentage missing.  
Ways we could deal with missing values:
- Permutations. For n observations missing in the column, shuffle the available values and assign randomly with OR without replacement to fill n values
- Mean, mode, max, min, 25%, 50% (Median), 75% percentiles
- Leave as -1 for missing or convert to NaN to keep it out of numerical consideration

Using the mean for float(continuous) and the mode for the non-categorical/int(discrete) columns would result in a strengthening of the distribution for those columns rather than skewing it away from the mean. We can't use mean for categorical columns because they need to contain discrete values. We could use mode/median for categorical columns. Imputing will introduce bias to the data.

With the remaining missing values, we will apply the following rule for imputation:
- If it is a discrete-valued column, and not binary, replace with mode
    - ps_ind_02_cat, ps_ind_04_cat, ps_ind_05_cat, ps_car_01_cat, ps_car_02_cat, ps_car_07_cat, ps_car_09_cat, ps_car_11
- If it is in a continuous-valued column, replace with median
    - ps_reg_03, ps_car_12, ps_car_14
    

In [14]:
#Dropping columns with > 35% missing values
train.drop(['ps_car_03_cat','ps_car_05_cat'], axis = 1)
test.drop(['ps_car_03_cat','ps_car_05_cat'], axis = 1)

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_04_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,0,0,0,0,0,12,1,0,0,0.5,0.3,0.610328,7,1,0,1,1,1,2,1,65,1,0.316228,0.669556,0.352136,3.464102,0.1,0.8,0.6,1,1,6,3,6,2,9,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,0,0,0,0,0,5,1,0,0,0.9,0.5,0.771362,4,1,0,11,1,1,0,1,103,1,0.316228,0.606320,0.358329,2.828427,0.4,0.5,0.4,3,3,8,4,10,2,7,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,0,0,0,0,0,10,0,0,0,0.4,0.0,0.916174,11,1,0,14,1,1,2,1,29,3,0.400000,0.896239,0.398497,3.316625,0.6,0.6,0.6,2,3,7,4,6,3,12,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,0,0,0,0,0,4,1,0,0,0.1,0.2,-1.000000,7,1,0,1,1,1,2,1,40,2,0.374166,0.652110,0.381445,2.449490,0.1,0.5,0.5,2,1,7,3,12,1,13,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,0,0,0,0,0,4,1,0,0,0.9,0.4,0.817771,11,1,0,11,1,1,2,1,101,3,0.374166,0.812914,0.385097,3.316625,0.9,0.6,0.8,3,4,7,1,10,4,12,4,0,0,4,0,1,1,0,0,1
5,5,0,1,6,0,0,1,0,0,0,0,0,0,0,0,10,1,0,0,0.9,0.5,0.759934,9,1,0,11,0,0,2,1,11,2,0.316228,0.750922,0.368782,3.605551,0.7,0.9,0.4,2,1,9,5,9,4,12,8,1,4,9,1,0,1,0,1,0
6,6,0,1,3,0,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.1,0.1,-1.000000,6,1,0,1,1,1,0,1,10,2,0.316228,0.724569,0.381445,3.316625,0.9,0.8,0.8,1,1,7,3,9,5,6,2,0,4,6,1,1,0,0,0,0
7,8,0,1,0,0,0,1,0,0,0,0,0,0,0,0,7,1,0,0,0.9,1.1,1.145917,7,1,0,11,1,1,2,1,103,3,0.316228,0.586610,0.358329,2.645751,0.8,0.9,0.5,2,2,8,4,11,2,8,3,1,4,9,0,1,0,0,0,0
8,10,0,1,7,0,0,0,1,0,0,0,0,0,0,0,6,1,0,0,0.4,0.0,0.597390,11,0,1,2,-1,0,0,1,104,2,0.447214,1.640147,0.367423,3.741657,0.9,0.3,0.0,2,2,9,7,9,0,10,5,1,4,6,0,0,1,0,0,0
9,11,1,1,6,0,0,0,0,0,1,0,0,0,0,0,7,0,1,0,0.9,1.0,-1.000000,11,0,0,4,1,1,2,1,104,2,0.447214,1.041336,0.422493,3.464102,0.0,0.9,0.7,2,1,7,1,9,1,11,6,1,6,10,0,1,1,0,0,0


# Preprocessing: Imputation on missing values

In [15]:
#can create metadata datatable to make this task easier
import re
pattern = re.compile('\w*cat\w*')
col_list = train.columns
print(col_list)

cat_vars = []
for col in col_list:
    if pattern.match(str(col)):
        cat_vars.append(col)
print(cat_vars)

bin_vars = []
pattern = re.compile('\w*bin\w*')
for col in col_list:
    if pattern.match(str(col)):
        bin_vars.append(col)
print(bin_vars)

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [16]:
#this needs to be done for the test set as well - Join train + test then dummify using pd.get_dummies

#Now we need to replace these missing values for train with the impute functionality of sklearn
mode_imp = Imputer(missing_values=-1, strategy = 'most_frequent', axis = 0)
median_imp = Imputer(missing_values=-1, strategy = 'median', axis=0)
train['ps_ind_02_cat'] = mode_imp.fit_transform(train[['ps_ind_02_cat']]).ravel()
train['ps_ind_04_cat'] = mode_imp.fit_transform(train[['ps_ind_04_cat']]).ravel()
train['ps_ind_05_cat'] = mode_imp.fit_transform(train[['ps_ind_05_cat']]).ravel()
train['ps_reg_03'] = median_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_01_cat'] = mode_imp.fit_transform(train[['ps_car_01_cat']]).ravel()
train['ps_car_02_cat'] = mode_imp.fit_transform(train[['ps_car_02_cat']]).ravel()
train['ps_car_07_cat'] = mode_imp.fit_transform(train[['ps_car_07_cat']]).ravel()
train['ps_car_09_cat'] = mode_imp.fit_transform(train[['ps_car_09_cat']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()
train['ps_car_12'] = median_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = median_imp.fit_transform(train[['ps_car_12']]).ravel()
#ravel makes them into arrays

#same thing for test
test['ps_ind_02_cat'] = mode_imp.fit_transform(test[['ps_ind_02_cat']]).ravel()
test['ps_ind_04_cat'] = mode_imp.fit_transform(test[['ps_ind_04_cat']]).ravel()
test['ps_ind_05_cat'] = mode_imp.fit_transform(test[['ps_ind_05_cat']]).ravel()
test['ps_reg_03'] = median_imp.fit_transform(test[['ps_reg_03']]).ravel()
test['ps_car_01_cat'] = mode_imp.fit_transform(test[['ps_car_01_cat']]).ravel()
test['ps_car_02_cat'] = mode_imp.fit_transform(test[['ps_car_02_cat']]).ravel()
test['ps_car_07_cat'] = mode_imp.fit_transform(test[['ps_car_07_cat']]).ravel()
test['ps_car_09_cat'] = mode_imp.fit_transform(test[['ps_car_09_cat']]).ravel()
test['ps_car_11'] = mode_imp.fit_transform(test[['ps_car_11']]).ravel()
test['ps_car_12'] = median_imp.fit_transform(test[['ps_car_12']]).ravel()
test['ps_car_14'] = median_imp.fit_transform(test[['ps_car_12']]).ravel()

Now that we've dealt with the missing values, we should turn our attention to the categorical variables. First, we will check how many distinct values there are within each categorical variable, and then we will dummify them as a further preprocessing step.

In [17]:
print(train.shape[0])
#train rows end at index 595212

x_train = train.drop(['id','target'],axis=1)
x_test = test.drop(['id'],axis=1)
y = train['target']

df_merged = x_train.append(x_test)
print(df_merged[595210:595215])


#_______________________________documentation_______________________
#it indexes 
#cat_cols = []
#for col in train.columns:
#    if 'cat' in col:
#        #cat_cols.append(col)
#        train[col] = train[col].astype(str)
#df_dummies_train = pd.get_dummies(train,drop_first=True)
#print(df_dummies_train.columns)


#we have to do the same for the test data
#for col in test.columns:
#    if 'cat' in col:
#        test[col] = test[col].astype(str)
#df_dummies_test = pd.get_dummies(test,drop_first=True)
#print(df_dummies_test.columns)


595212
        ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  ps_ind_05_cat  \
595210          5            2.0          3            1.0            0.0   
595211          0            1.0          8            0.0            0.0   
0               0            1.0          8            1.0            0.0   
1               4            2.0          5            1.0            0.0   
2               5            1.0          3            0.0            0.0   

        ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  ps_ind_09_bin  \
595210              0              0              1              0   
595211              1              0              0              0   
0                   0              1              0              0   
1                   0              0              0              1   
2                   0              0              0              1   

        ps_ind_10_bin  ps_ind_11_bin  ps_ind_12_bin  ps_ind_13_bin  ps_ind_14  \
595210              0       

In [18]:
#I am setting drop_first to false to avoid loss of data for
#when we split up the merged dataframe back into train_dummies/test_dummies
for col in df_merged.columns:
    if 'cat' in col:
        df_merged[col] = df_merged[col].astype(str)
df_dummies = pd.get_dummies(df_merged,drop_first=False)
print(df_dummies.columns)

x_train = df_dummies[0:595212]
x_test = df_dummies[595212:]

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       'ps_car_11_cat_90', 'ps_car_11_cat_91', 'ps_car_11_cat_92',
       'ps_car_11_cat_93', 'ps_car_11_cat_94', 'ps_car_11_cat_95',
       'ps_car_11_cat_96', 'ps_car_11_cat_97', 'ps_car_11_cat_98',
       'ps_car_11_cat_99'],
      dtype='object', length=220)


In [19]:
pattern = re.compile('\w*cat\w*')
col_list = x_train.columns
print(col_list)

cat_vars_dummies = []
for col in col_list:
    if pattern.match(str(col)):
        cat_vars_dummies.append(col)
print(cat_vars_dummies)

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       'ps_car_11_cat_90', 'ps_car_11_cat_91', 'ps_car_11_cat_92',
       'ps_car_11_cat_93', 'ps_car_11_cat_94', 'ps_car_11_cat_95',
       'ps_car_11_cat_96', 'ps_car_11_cat_97', 'ps_car_11_cat_98',
       'ps_car_11_cat_99'],
      dtype='object', length=220)
['ps_ind_02_cat_1.0', 'ps_ind_02_cat_2.0', 'ps_ind_02_cat_3.0', 'ps_ind_02_cat_4.0', 'ps_ind_04_cat_0.0', 'ps_ind_04_cat_1.0', 'ps_ind_05_cat_0.0', 'ps_ind_05_cat_1.0', 'ps_ind_05_cat_2.0', 'ps_ind_05_cat_3.0', 'ps_ind_05_cat_4.0', 'ps_ind_05_cat_5.0', 'ps_ind_05_cat_6.0', 'ps_car_01_cat_0.0', 'ps_car_01_cat_1.0', 'ps_car_01_cat_10.0', 'ps_car_01_cat_11.0', 'ps_car_01_cat_2.0', 'ps_car_01_cat_3.0', 'ps_car_01_cat_4.0', 'ps_car_01_cat_5.0', 'ps_car_01_cat_6.0', 'ps_car_01_cat_7.0', 'ps_car_01_cat_8.0', 'ps_car_01_cat_9.0', 'ps_car_02_cat_

# Model Testing and Evaluation
## Model 1: Logistic Regression with no parameters, no scaling

logreg = LogisticRegression()
logreg.fit(x_train, y)
y_pred = logreg.predict_proba(x_test)[:,1]
print(y_pred)
logreg_1 = pd.DataFrame({'id':test['id'], 'target':y_pred})
logreg_1.set_index(['id'], drop=True, inplace=True)
print(logreg)
logreg_1.to_csv('logreg', sep=',')
#scores .25618 public, .26023 private


# Baseline tests with target as 0, .5, and 1. 

## 0% accuracy, scored negative on test.

In [20]:
#Using logreg variable as the empty shell to fill in an answer set
#baseline_0s = logreg
#baseline_0s['target'] = 0
#baseline_0s.to_csv('baseline_0s', sep=',')

#baseline_half = logreg
#baseline_half['target'] = .5
#baseline_half.to_csv('baseline_half', sep=',')

#baseline_1s = logreg
#baseline_1s['target'] = 1
#baseline_1s.to_csv('baseline_1s', sep=',')

#baseline_randoms = logreg
#baseline_randoms['target'] = np.random.rand(892816, 1)
#baseline_randoms.to_csv('baseline_randoms', set=',')

scaler = StandardScaler()

scale_set = x_train.drop(bin_vars, axis = 1)
scale_set = scale_set.drop(cat_vars, axis=1)
scale_result = scaler.fit_transform(scale_set.values)

scale_set_df = pd.DataFrame(scale_set, index = scale_set.index,columns = scale_set.columns)

print(type(scale_set))
print(scale_set.shape)
scaled_set = pd.DataFrame(scale_set)

df_cat = pd.DataFrame(x_train[cat_vars])
df_bin = pd.DataFrame(x_train[bin_vars])
scaled_train_x = pd.concat([scaled_set, df_cat], axis = 1)
scaled_train_x = pd.concat([scaled_train_x, df_bin], axis = 1)
print(scaled_train_x.shape)
print(scaled_train_x)

# Using scaler on x_train, x_test

## note: only using scaler on the non-categorical/binary columns

In [21]:
scaler = StandardScaler()
var_scale = []
for col in x_train.columns:
    var_scale.append(col)
    
drop_vars = cat_vars_dummies + bin_vars
for var in drop_vars:
    var_scale.remove(var)
x_train_scaled = x_train
x_test_scaled = x_test
print(var_scale)
x_train_scaled[var_scale] = scaler.fit_transform(x_train[var_scale])
x_test_scaled[var_scale] = scaler.fit_transform(x_test[var_scale])
print(x_train_scaled)

['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14']


C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Owner\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

        ps_ind_01  ps_ind_03  ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  \
0        0.050218   0.213594              0              1              0   
1       -0.453868   0.954362              0              0              1   
2        1.562477   1.695130              0              0              1   
3       -0.957955  -0.897559              1              0              0   
4       -0.957955  -1.638327              1              0              0   
5        1.562477  -0.156790              0              0              0   
6        0.050218  -0.527174              0              1              0   
7        1.562477  -0.156790              1              0              0   
8        1.562477  -0.527174              0              0              1   
9       -0.453868  -0.897559              0              1              0   
10       1.562477  -0.897559              0              0              1   
11       0.050218  -0.527174              0              1              0   

In [22]:
print(x_test.shape)

(892816, 220)


# Model 2: Logistic Regression with StandardScaler()

#what models am I going to try? I would use KNN first because I don't know neural networks yet.

#look into Chi-squared feature selector for KNN because it is very slow with high dimensions
#removing features (heavy correlation)

#logistic regression is best baseline model

#model = RandomForestClassifier()

#KNN is very slow with high dimension, maybe try logistic regression for a faster output?

#i want to test p = 1 (manhattan distance L1) vs p = 2 (euclidian distance L2)
#knn = KNeighborsClassifier(n_neighbors = 2, p = 2)

logreg = LogisticRegression()
logreg.fit(x_train_scaled, y)
y_pred = logreg.predict_proba(x_test_scaled)[:,1]
print(y_pred)

logreg_scaled= pd.DataFrame({'id':test['id'], 'target':y_pred})
logreg_scaled.set_index(['id'], drop=True, inplace=True)
print(logreg_1)
logreg_1.to_csv('logreg_scaled', sep=',')
#public .25617, private .26022

Things we can do to improve the model:
- standard scaling
- using different imputation methods
- changing distance metric
- dimensionality reduction PCA
    

EDA
feature relation to target variable
features vs features
distributions
inferential stats (hypothesis testing, distribution of variables)
sklearn gradientboostingclassifier()

print(y_pred.shape)

In [23]:
print(x_test.shape)

(892816, 220)


In [24]:
print(df_dummies.shape)
z = df_dummies[595212:]
print(z.shape)

(1488028, 220)
(892816, 220)


In [25]:
print(np.sum(y))

21694


# Model 3: RandomForestClassifier without scaling

#tune this by parameters and see if the results get better
rf1 = RandomForestClassifier()
rf1.fit(x_train, y)
y_pred = rf1.predict_proba(x_test)[:,1]

submission2 = pd.DataFrame({'id':test['id'], 'target':y_pred})
submission2.set_index(['id'], drop=True, inplace=True)
submission2.to_csv('RandomForest', sep=',')
# public .05504 private .06802

# Model 4: RandomForestClassifier with scaling

rf2 = RandomForestClassifier()
rf2.fit(x_train_scaled, y)
y_pred = rf2.predict_proba(x_test_scaled)[:,1]

submission3 = pd.DataFrame({'id':test['id'], 'target':y_pred})
submission3.set_index(['id'], drop=True, inplace=True)
submission3.to_csv('RandomForest_scaled', sep=',')
#Private 0.06449
#Public 0.0625

# Model 5: RandomForestClassifier with specific parameters and no scaling

rf3 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf3.fit(x_train, y)
y_pred = rf3.predict_proba(x_test)[:,1]

RFC_param_set_1 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_param_set_1.set_index(['id'], drop=True, inplace=True)
RFC_param_set_1.to_csv('RFC_parameter_set_1', sep=',')
#public .25915
#private .26199 *Rank 1 solution 7/1/18.

# Model 6: RandomForestClassifier with parameter set 1 and scaling

rf4 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf4.fit(x_train_scaled, y)
y_pred = rf4.predict_proba(x_test_scaled)[:,1]

RFC_param_set_1_scaled = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_param_set_1_scaled.set_index(['id'], drop=True, inplace=True)
RFC_param_set_1_scaled.to_csv('RFC_parameter_set_1_scaled', sep=',')
#public .25895
#private .26220 #highest private score



The public data given for this project accounts for 70% of the score, whereas the other 30% is an unreleased private set. 

Model 6 (scaled data set) scored the highest in the private set however performed worse on the public set than Model 5 (unscaled data set), although only by a tiny margin. 

Scoring high in the private set equates to having a robust model that is not only fit well to the train set. ROC curve? 

***Monte carlo simulation: 5 iterations running the model with 5 random parameter sets of Model 5 vs Model 6 to test scaled vs unscaled data


for i in [300, 600, 900, 1200]:
    for j in [1,5,7]:
        RCF = RandomForestClassifier(n_estimators = i, max_depth = j)
        df = pd.DataFrame({'id':test['id'], 'target':y_pred})
        df.set_index(['id'], drop=True, inplace=True)
        df.to_csv('RFC(%d)' % i, sep=',')
    


print(max(rf4.feature_importances_))
print(sum(rf4.feature_importances_))
print(rf4.feature_importances_.shape)


features_list = x_train.columns
maxIndex = np.argmax(rf4.feature_importances_)

print(features_list[maxIndex-1])
print(rf4.feature_importances_[maxIndex])

maximum value feature is 7.23% influence. if we were to take this column out, there should be a presumably lower score. 
Also, if we were to base the model upon the top "n" scores we can perhaps achieve a better score without the "clutter variables"

Next, check chisquare test for more feature analysis

chi_y = np.vstack([1 - y, y])
observed = np.dot(chi_y, x_train)


feature_count = x_train_scaled.sum(axis=0)
class_prob = chi_y.mean(axis=1)
expected = np.dot(feature_count.reshape(-1,1), class_prob.reshape(1,-1)).T
score, pval = chisquare(observed, expected)
print(max(score))
print(score)

# Cross verifications 5-fold
## train test split and Cross validation for hyperparameter tuning without scaling

In [26]:
#iteration 1
# 'n_estimators':[120, 300, 500, 800, 1000], 'max_depth':[5,8,15,25,30], 'min_samples_split':[2,5,10,15,50], 'min_samples_leaf':[1,2,5,10]
#I left my computer on while sleeping and it did not execute the code

#train/test/split
#we were given a training set and a testing set by the customer which gave us x_train and y, which is the training target. 
#we are going to split up x_train and y using 5-fold Cross Validation to find the best parameters

X_train, X_test, y_train, y_test = train_test_split(x_train, y, test_size=0.3, random_state = 0)
#param_grid = {'n_estimators':[120], 'max_depth':[5,8], 'min_samples_split':[2,5], 'min_samples_leaf':[1,2]}
param_grid = {'n_estimators':[120, 300],
              'max_depth':[5,8],
              'min_samples_split':[2,5],
              'min_samples_leaf':[1,2]}

model = RandomForestClassifier()
grid_cv = GridSearchCV(model, param_grid, cv = 5)
grid_cv.fit(X_train,y_train)

print("Best parameters set found")
print(grid_cv.best_params_)

## train test split and Cross validation for hyperparameter tuning with scaling

#train/test/split
#we were given a training set and a testing set by the customer which gave us x_train and y, which is the training target. 
#we are going to split up x_train and y using 5-fold Cross Validation to find the best parameters

X_train, X_test, y_train, y_test = train_test_split(x_train_scaled, y, test_size=0.3, random_state = 0)
#param_grid = {'n_estimators':[120], 'max_depth':[5,8], 'min_samples_split':[2,5], 'min_samples_leaf':[1,2]}
param_grid = {'n_estimators':[120, 300],
              'max_depth':[5,8],
              'min_samples_split':[2,5],
              'min_samples_leaf':[1,2]}

model = RandomForestClassifier()
grid_cv = GridSearchCV(model, param_grid, cv = 5)
grid_cv.fit(X_train,y_train)

print("Best parameters set found")
print(grid_cv.best_params_)

# Testing the hyperparameters first against unscaled data, then scaled

rf5 = RandomForestClassifier(n_estimators = 120, max_depth = 5, min_samples_split =2, min_samples_leaf = 1)
rf5.fit(x_train, y)
y_pred = rf5.predict_proba(x_test)[:,1]
RFC_CV5 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_CV5.set_index(['id'], drop=True, inplace=True)
RFC_CV5.to_csv('RFC_CV5', sep=',')
#score 0.24540 public 0.24919 private

rf6 = RandomForestClassifier(n_estimators = 120, max_depth = 5, min_samples_split =2, min_samples_leaf = 1)
rf6.fit(x_train, y)
y_pred = rf6.predict_proba(x_test)[:,1]
RFC_CV5_scaled = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_CV5_scaled.set_index(['id'], drop=True, inplace=True)
RFC_CV5_scaled.to_csv('RFC_CV5_scaled', sep=',')

# These results are far below the logistic regression model so instead of trying to tune the RandomForestClassifier() with CVGridSearch, tuning the linear regression model will produce an improvement upon the already 90% solution as mentioned above.

In [27]:
C = np.arange(.0001,8,1)

# Grid_CV for logistic regression using scaled training set

#trying to find out if the parameters change with more estimators
param_grid = {'penalty':['l1', 'l2'],
             'C':C}

logreg = LogisticRegression()
grid_cv = GridSearchCV(logreg, param_grid, cv = 5)
grid_cv.fit(X_train,y_train)


#Best parameters set found
#{'C': 0.0001, 'penalty': 'l1'}

print("Best parameters set found")
print(grid_cv.best_params_)

# Fitting to best parameters

#I tried to fit to X which had standard scaler applied to it... it did not work.
logreg_best = LogisticRegression(penalty = 'l1', C = .0001)
logreg_best.fit(x_train_scaled, y)
y_pred = logreg_best.predict_proba(x_test_scaled)[:,1]


logreg_cv5 = pd.DataFrame({'id':test['id'], 'target':y_pred})
logreg_cv5.set_index(['id'], drop=True, inplace=True)
logreg_cv5.to_csv('logreg_cv5', sep=',')
#0.02694 score, very bad

# Chi-Squared Test

## separating pd.get_dummies() variables

In [28]:
#v1 = np.vstack([1 - y, y])
#observed = np.dot(v1, X)


#feature_count = X.sum(axis=0)
#class_prob = Y.mean(axis=1)
#expected = np.dot(feature_count.reshape(-1, 1), class_prob.reshape(1, -1)).T

X = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat']

train.drop(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat',], axis = 1)

# defining the Chi Square function

In [29]:
class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)

    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)
        



In [30]:
print(x_train.loc[:, cat_vars_dummies])

        ps_ind_02_cat_1.0  ps_ind_02_cat_2.0  ps_ind_02_cat_3.0  \
0                       0                  1                  0   
1                       1                  0                  0   
2                       0                  0                  0   
3                       1                  0                  0   
4                       0                  1                  0   
5                       1                  0                  0   
6                       1                  0                  0   
7                       1                  0                  0   
8                       1                  0                  0   
9                       1                  0                  0   
10                      0                  1                  0   
11                      1                  0                  0   
12                      1                  0                  0   
13                      1                  0                  

In [31]:
print(type(cat_vars_dummies))
cat_vars_dummies = x_train.loc[:,cat_vars_dummies].columns
print(type(cat_vars_dummies))


#dummy_train = x_train.join(y)
dummy_train = pd.concat([x_train_scaled, y], axis=1)
print(dummy_train)

<class 'list'>
<class 'pandas.core.indexes.base.Index'>
        ps_ind_01  ps_ind_03  ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  \
0        0.050218   0.213594              0              1              0   
1       -0.453868   0.954362              0              0              1   
2        1.562477   1.695130              0              0              1   
3       -0.957955  -0.897559              1              0              0   
4       -0.957955  -1.638327              1              0              0   
5        1.562477  -0.156790              0              0              0   
6        0.050218  -0.527174              0              1              0   
7        1.562477  -0.156790              1              0              0   
8        1.562477  -0.527174              0              0              1   
9       -0.453868  -0.897559              0              1              0   
10       1.562477  -0.897559              0              0              1   
11       0.050218  -

In [32]:
pattern = re.compile('\w*cat\w*')
col_list = train.columns


cat_vars = []
for col in col_list:
    if pattern.match(str(col)):
        cat_vars.append(col)


bin_vars = []
pattern = re.compile('\w*bin\w*')
for col in col_list:
    if pattern.match(str(col)):
        bin_vars.append(col)

nominal_vars = []
for var in cat_vars:
    nominal_vars.append(var)
for var in bin_vars:
    nominal_vars.append(var)
print(nominal_vars)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']


In [33]:
df = train.drop(['id'], axis = 1)

#initializing the ChiSquare class
chi_init = ChiSquare(df)

#Feature Selection
for var in nominal_vars:
    chi_init.TestIndependence(colX=var, colY='target')

#now going to use the dataframe with dummified variables to see what happens
#chi_init_dummies = ChiSquare(dummy_train)
#print("\n\n")
#for var in cat_vars_dummies:
    #chi_init_dummies.TestIndependence(colX=var, colY='target')
#this doesn't make sense because the premise of ChiSquared testing is that the features must be independent of one another
#Dummy variables are all dependent on one another.

ps_ind_02_cat is IMPORTANT for Prediction
ps_ind_04_cat is IMPORTANT for Prediction
ps_ind_05_cat is IMPORTANT for Prediction
ps_car_01_cat is IMPORTANT for Prediction
ps_car_02_cat is IMPORTANT for Prediction
ps_car_03_cat is IMPORTANT for Prediction
ps_car_04_cat is IMPORTANT for Prediction
ps_car_05_cat is IMPORTANT for Prediction
ps_car_06_cat is IMPORTANT for Prediction
ps_car_07_cat is IMPORTANT for Prediction
ps_car_08_cat is IMPORTANT for Prediction
ps_car_09_cat is IMPORTANT for Prediction
ps_car_10_cat is NOT an important predictor. (Discard ps_car_10_cat from model)
ps_car_11_cat is IMPORTANT for Prediction
ps_ind_06_bin is IMPORTANT for Prediction
ps_ind_07_bin is IMPORTANT for Prediction
ps_ind_08_bin is IMPORTANT for Prediction
ps_ind_09_bin is IMPORTANT for Prediction
ps_ind_10_bin is NOT an important predictor. (Discard ps_ind_10_bin from model)
ps_ind_11_bin is NOT an important predictor. (Discard ps_ind_11_bin from model)
ps_ind_12_bin is IMPORTANT for Prediction
ps_i

# Feature analysis results: Removing low significance predictors from model

After we remove the indicated features, we will test some of the previously tested predictor models to see if it makes a difference. We will have to delete all traces of the dummy variables of these features within x_train_scaled

In [34]:
remove_nominals = ['ps_car_10_cat', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_calc_15_bin', 
                   'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']
nominal_dummies = []
col_list = dummy_train.columns
#pattern = re.compile('\w*ps_car_10_cat\w*')
for var in remove_nominals:
    pattern = re.compile('\w*%s\w*' %var)
    for col in col_list:
        if pattern.match(str(col)):
            nominal_dummies.append(col)
print(nominal_dummies)

['ps_car_10_cat_0', 'ps_car_10_cat_1', 'ps_car_10_cat_2', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']


The result above makes sense because binary variables do not need dummies. For instance, if the variable is whether the person is a male or female, then having a 1 or a 0 as the value covers both possibilities and there is no need for any further variables. 

In [35]:
print(dummy_train.columns)

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       'ps_car_11_cat_91', 'ps_car_11_cat_92', 'ps_car_11_cat_93',
       'ps_car_11_cat_94', 'ps_car_11_cat_95', 'ps_car_11_cat_96',
       'ps_car_11_cat_97', 'ps_car_11_cat_98', 'ps_car_11_cat_99', 'target'],
      dtype='object', length=221)


In [36]:
x_train_reduced = dummy_train.drop(nominal_dummies, axis = 1)
x_train_reduced.drop(['target'],axis=1, inplace = True)
#need to also do it for the test set
x_test_reduced = x_test_scaled.drop(nominal_dummies, axis = 1)
print(x_train_reduced.columns)
print(x_test_reduced.columns)

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin', 'ps_ind_14',
       'ps_ind_15', 'ps_ind_16_bin',
       ...
       'ps_car_11_cat_90', 'ps_car_11_cat_91', 'ps_car_11_cat_92',
       'ps_car_11_cat_93', 'ps_car_11_cat_94', 'ps_car_11_cat_95',
       'ps_car_11_cat_96', 'ps_car_11_cat_97', 'ps_car_11_cat_98',
       'ps_car_11_cat_99'],
      dtype='object', length=208)
Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin', 'ps_ind_14',
       'ps_ind_15', 'ps_ind_16_bin',
       ...
       'ps_car_11_cat_90', 'ps_car_11_cat_91', 'ps_car_11_cat_92',
       'ps_car_11_cat_93', 'ps_car_11_cat_94', 'ps_car_11_cat_95',
       'ps_car_11_cat_96', 'ps_car_11_cat_97', 'ps_car_11_cat_98',
       'ps_car_11_cat_99'],
      dtype='object', length=208)


In [39]:
print(x_train.columns)


Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin',
       ...
       'ps_car_11_cat_90', 'ps_car_11_cat_91', 'ps_car_11_cat_92',
       'ps_car_11_cat_93', 'ps_car_11_cat_94', 'ps_car_11_cat_95',
       'ps_car_11_cat_96', 'ps_car_11_cat_97', 'ps_car_11_cat_98',
       'ps_car_11_cat_99'],
      dtype='object', length=220)


# Running the adjusted model: starting basic with logistic regression

In [41]:
logreg = LogisticRegression()
logreg.fit(x_train_reduced, y)
y_pred = logreg.predict_proba(x_test_reduced)[:,1]
print(y_pred)
logreg_reduced_1 = pd.DataFrame({'id':test['id'], 'target':y_pred})
logreg_reduced_1.set_index(['id'], drop=True, inplace=True)
print(logreg)
logreg_reduced_1.to_csv('logreg_dummy_reduced', sep=',')


[ 0.02838188  0.03285152  0.02502448 ...,  0.03971578  0.02356938
  0.03307927]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [42]:
#running with unscaled data for comparison purposes to remain consistent
x_train_adj = x_train.drop(nominal_dummies, axis = 1)
x_test_adj = x_test.drop(nominal_dummies, axis = 1)
logreg = LogisticRegression()
logreg.fit(x_train_adj, y)
y_pred = logreg.predict_proba(x_test_adj)[:,1]
print(y_pred)
logreg_adj_1 = pd.DataFrame({'id':test['id'], 'target':y_pred})
logreg_adj_1.set_index(['id'], drop=True, inplace=True)
print(logreg)
logreg_reduced_1.to_csv('logreg_adj_1', sep=',')


[ 0.02838188  0.03285152  0.02502448 ...,  0.03971578  0.02356938
  0.03307927]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


# Results and comparison of previous model to adjusted model 

1) scores unscaled data: public,  private(previously .25618, .26023)
    - Public = .25632
    - Private = .26018
2) By removing 1 feature: ps_car_cat_10, and using scaled data: 
    - Public = .25607
    - Private = .26014

3) By removing all features that did not pass the ChiSquared test with alpha = 5%,
    - Public = .25632
    - Private = .26018
    - This is an improvement in the public score with all the features from .25618 to .25632
    - This is a decrease in the private score from .26023 to .26018
    - Overall, an increase in the public score means the model is trained better to the train data set but did not turn out to be more versatile/robust against predicting against unseen data. This means that the adjusted model performed as expected theoretically.
    - Notice this score is exactly the same as the unscaled data set.

# Now to try with RFC

In [43]:
rf5 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf5.fit(x_train_reduced, y)
y_pred = rf5.predict_proba(x_test_reduced)[:,1]

RFC_param_set_1_scaled_reduced = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_param_set_1_scaled_reduced.set_index(['id'], drop=True, inplace=True)
RFC_param_set_1_scaled_reduced.to_csv('RFC_parameter_set_1_scaled_reduced', sep=',')

1) By removing 1 feature: ps_car_cat_10, and using scaled data: 
    - public 0.25897 (previously .25895)
    - private 0.26198 (previously .26220 highest private score)
    
2) By removing all of the above features deemed insignificant, and using scaled data:
    - public .25919 (new best score)
    - private .26241 (new best score)

# Another way to prioritize features is to take the top n features of importance

In [44]:
print(max(rf5.feature_importances_))
print(sum(rf5.feature_importances_))
print(rf5.feature_importances_.shape)


features_list = x_train.columns
maxIndex = np.argmax(rf5.feature_importances_)

print(features_list[maxIndex-1])
print(rf5.feature_importances_[maxIndex])

#getting indices of top 5 influential features
top_5 = rf5.feature_importances_.argsort()[-5:][::-1]
print(top_5)
#printing percentages of top 5
cumulative_influence = 0
for i in top_5:
    cumulative_influence += rf5.feature_importances_[i]
print(cumulative_influence)

0.0752819254351
1.0
(208,)
ps_reg_02
0.0752819254351
[17 14  1 13 10]
0.228466190766


The top 5 only accounts for 22.2% of the predictive influence. The goal can be augmented by setting an arbitrary percentage influence and taking those features into account rather than coming up with a list of numbers.

- Try top features accounting for 50%, 75%, 90%, and 95%

In [73]:
sorted_features = rf5.feature_importances_.argsort()[::-1]
track_50 = 0
list_50 = []
while track_50 <= .50:
    for i in sorted_features:
        track_50 += rf5.feature_importances_[i]
        list_50.append(i)
        if track_50 >= .5:
            break
print(list_50)
print(track_50)

sorted_features = rf5.feature_importances_.argsort()[::-1]
track_75 = 0
list_75 = []
while track_75 <= .75:
    for i in sorted_features:
        track_75 += rf5.feature_importances_[i]
        list_75.append(i)
        if track_75 >=.75:
            break
print(list_75)
print(track_75)

sorted_features = rf5.feature_importances_.argsort()[::-1]
track_90 = 0
list_90 = []
while track_90 <= .90:
    for i in sorted_features:
        track_90 += rf5.feature_importances_[i]
        list_90.append(i)
        if track_90 >= .90:
            break
print(list_90)
print(track_90)

sorted_features = rf5.feature_importances_.argsort()[::-1]
track_95 = 0
list_95 = []
while track_95 <= .95:
    for i in sorted_features:
        track_95 += rf5.feature_importances_[i]
        list_95.append(i)
        if track_95 >= .95:
            break
print(list_95)
print(track_95)

[17, 14, 1, 13, 10, 40, 18, 16, 8, 19, 29, 30, 33, 12, 3, 0, 2, 32, 26]
0.513529793399
[17, 14, 1, 13, 10, 40, 18, 16, 8, 19, 29, 30, 33, 12, 3, 0, 2, 32, 26, 64, 9, 21, 27, 20, 22, 28, 63, 24, 25, 31, 61, 46, 23, 100, 59, 60, 56, 96, 42]
0.750341149982
[17, 14, 1, 13, 10, 40, 18, 16, 8, 19, 29, 30, 33, 12, 3, 0, 2, 32, 26, 64, 9, 21, 27, 20, 22, 28, 63, 24, 25, 31, 61, 46, 23, 100, 59, 60, 56, 96, 42, 15, 95, 74, 44, 50, 58, 73, 98, 97, 4, 38, 5, 76, 39, 7, 75, 99, 101, 34, 11, 35, 62, 94, 110, 66, 41, 53, 84, 37, 144, 65, 49, 55, 86, 57, 80, 90, 45, 36, 103, 43, 6, 77, 54]
0.900335813592
[17, 14, 1, 13, 10, 40, 18, 16, 8, 19, 29, 30, 33, 12, 3, 0, 2, 32, 26, 64, 9, 21, 27, 20, 22, 28, 63, 24, 25, 31, 61, 46, 23, 100, 59, 60, 56, 96, 42, 15, 95, 74, 44, 50, 58, 73, 98, 97, 4, 38, 5, 76, 39, 7, 75, 99, 101, 34, 11, 35, 62, 94, 110, 66, 41, 53, 84, 37, 144, 65, 49, 55, 86, 57, 80, 90, 45, 36, 103, 43, 6, 77, 54, 47, 72, 79, 52, 78, 106, 102, 181, 48, 69, 82, 83, 122, 201, 85, 91, 205, 1

In [74]:
print(len(list_50))
print(len(list_75))
print(len(list_90))
print(len(list_95))

19
39
83
123


# Feature importances count
    - Top 50% of feature importance in model is 19 features
    - Top 75% of feature importance in model is 39 features
    - Top 90% of feature importance in model is 83 features
    - Top 95% of feature importance in model is 123 features

In [76]:
model_50_train = x_train_reduced.iloc[:, list_50]
model_50_test = x_test_reduced.iloc[:,list_50]

model_75_train = x_train_reduced.iloc[:, list_75]
model_75_test = x_test_reduced.iloc[:,list_75
                                   ]
model_90_train = x_train_reduced.iloc[:, list_90]
model_90_test = x_test_reduced.iloc[:,list_90]

model_95_train = x_train_reduced.iloc[:, list_95]
model_95_test = x_test_reduced.iloc[:,list_95]

In [70]:
rf_50 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf_50.fit(model_50_train, y)
y_pred = rf_50.predict_proba(model_50_test)[:,1]

RFC_50 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_50.set_index(['id'], drop=True, inplace=True)
RFC_50.to_csv('RFC_50', sep=',')

In [71]:
rf_75 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf_75.fit(model_75_train, y)
y_pred = rf_75.predict_proba(model_75_test)[:,1]

RFC_75 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_75.set_index(['id'], drop=True, inplace=True)
RFC_75.to_csv('RFC_75', sep=',')

In [72]:
rf_90 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf_90.fit(model_90_train, y)
y_pred = rf_90.predict_proba(model_90_test)[:,1]

RFC_90 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_90.set_index(['id'], drop=True, inplace=True)
RFC_90.to_csv('RFC_90', sep=',')

In [77]:
rf_95 = RandomForestClassifier(n_estimators = 1200, max_depth = 9)
rf_95.fit(model_95_train, y)
y_pred = rf_95.predict_proba(model_95_test)[:,1]

RFC_95 = pd.DataFrame({'id':test['id'], 'target':y_pred})
RFC_95.set_index(['id'], drop=True, inplace=True)
RFC_95.to_csv('RFC_95', sep=',')

# Results
    - Model_50 
        - Public = .24020
        - Private = .24557
        
    - Model_75
        - Public = .25611
        - Private = .25913
       
    - Model_90
        - Public = 0.26027 (highest public score)
        - Private = 0.26306 (highest private score)
    
    - Model_95
        - Public = 0.25995 
        - Private = .26330 (highest private score)

# Next steps... 1) remove calc variables to keep it all nominal and see
# 2) upsampling. make target variable 10-30% of total random sample taken
# 3) ROC curve of a given model as fitted to the train/test data
# 4) SKLEARN METRICS - accuracy, precision, recall
    - I want to be able to show improvements in accuracy/precision of the predictions can be made at least on train set
# 5) Clustering